In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + "202001-04_Trade_Support.xlsx"

ts_df = pd.read_excel(input_file_path, dtype=str)

ts_df.columns = ['YearMonth', 'SKU', 'Cost']
ts_df["Cost"] = ts_df["Cost"].astype("float")

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + "202001-05_brand_province_split.xlsx"

brand_prov_split = pd.read_excel(input_file_path, dtype=str)

brand_prov_split['Split_Ratio'] = brand_prov_split['Split_Ratio'].astype("float")

In [ ]:
ts_prov_df = pd.merge(ts_df, brand_prov_split, on=['YearMonth', 'SKU'], how='left')

ts_prov_df[ts_prov_df['Split_Ratio'] == np.NAN]

In [ ]:
ts_prov_df['Spending_Value'] = ts_prov_df['Cost'] * ts_prov_df['Split_Ratio']

ts_prov_df = ts_prov_df[['YearMonth', 'Province_Code', 'SKU', 'Spending_Value']]

ts_all_df = ts_prov_df.groupby(by = ['YearMonth', 'SKU'])['Spending_Value'].sum().reset_index()
ts_all_df['Province_Code'] = 'National'

ts_prov_df = ts_prov_df.append(ts_all_df, ignore_index=True)

In [ ]:
def format_yearmonth(year_month):
    return time.strftime("%Y/%m/%d", time.strptime(year_month,"%Y%m"))

def get_brand_name(brand):
    if brand == 'NC':
        return 'Nutrilon'
    elif (brand=='AC') | (brand == 'AP'):
        return "Aptamil"
    return ''

In [ ]:
ts_prov_df["Date"] = ts_prov_df['YearMonth'].apply(lambda x : format_yearmonth(x))

ts_prov_df["Brand_Name"] = ts_prov_df['SKU'].apply(
        lambda x: get_brand_name(x))

ts_prov_df['Cost_Type_1'] = 'Trade Support'
ts_prov_df['Cost_Type_2'] = ''
ts_prov_df['Cost_Type_3'] = ''
ts_prov_df['Cost_Type_4'] = ''
ts_prov_df['AIP'] = 'P'

ts_prov_df = ts_prov_df[['Date', 'Province_Code', 'Cost_Type_1', 'Cost_Type_2', 'Cost_Type_3', 'Cost_Type_4', 'AIP', 'Brand_Name', 'SKU', 'Spending_Value']]

In [ ]:
output_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + "202001-04_Trade_Support.xlsx"

ts_prov_df.to_excel(output_file_path, index=False)